In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import block
import ecc
import helper
import script
import tx

### Exercise 1

Write the `is_coinbase` method of the `Tx` class.

#### Make [this test](/edit/code-ch09/tx.py) pass: `tx.py:TxTest:test_is_coinbase`

In [2]:
# Exercise 1

# reload(tx)
# run(tx.TxTest("test_is_coinbase"))
def is_coinbase(self):
    if len(self.tx_ins) != 1:
        return False
    first_input = self.tx_ins[0]
    if first_input.prev_tx != b'\x00' * 32:
        return False
    if first_input.prev_index != 0xffffffff:
        return False
    return True

In [3]:
from io import BytesIO
from script import Script
stream = BytesIO(bytes.fromhex('4d04ffff001d0104455468652054696d65732030332f4a616e2f32303039204368616e63656c6c6f72206f6e206272696e6b206f66207365636f6e64206261696c6f757420666f722062616e6b73'))
s = Script.parse(stream)
print(s.cmds[2])


# return : b'The Times 03/Jan/2009 Chancellor on brink of second bailout for banks'

b'The Times 03/Jan/2009 Chancellor on brink of second bailout for banks'


In [4]:
from io import BytesIO
from script import Script
from helper import little_endian_to_int
stream = BytesIO(bytes.fromhex('5e03d71b07254d696e656420627920416e74506f6f6c20626a31312f4542312f4144362f43205914293101fabe6d6d678e2c8c34afc36896e7d9402824ed38e856676ee94bfdb0c6c4bcd8b2e5666a0400000000000000c7270000a5e00e00'))
script_sig = Script.parse(stream)
print(little_endian_to_int(script_sig.cmds[0]))

# return : 465879

465879


### Exercise 2

Write the `coinbase_height` method for the `Tx` class.

#### Make [this test](/edit/code-ch09/tx.py) pass: `tx.py:TxTest:test_coinbase_height`

In [5]:
# Exercise 2

# reload(tx)
# run(tx.TxTest("test_coinbase_height"))
def coinbase_height(self):
    if not self.is_coinbase():
        return None
    element = self.tx_ins[0].script_sig.cmds[0]
    return little_endian_to_int(element)

In [6]:
from helper import hash256
block_hash = hash256(bytes.fromhex('020000208ec39428b17323fa0ddec8e887b4a7c53b8c0a0a220cfd0000000000000000005b0750fce0a889502d40508d39576821155e9c9e3f5c3157f961db38fd8b25be1e77a759e93c0118a4ffd71d'))[::-1]
block_id = block_hash.hex()
print(block_id)

# return : 0000000000000000007e9e4c586439b0cdbe13b1370bdd9435d76a644d047523


0000000000000000007e9e4c586439b0cdbe13b1370bdd9435d76a644d047523


### Exercise 3

Write the `parse` for `Block`.

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_parse`

In [7]:
# Exercise 3

# reload(block)
# run(block.BlockTest("test_parse"))
def parse(cls, s):
    version = little_endian_to_int(s.read(4))
    prev_block = s.read(32)[::-1]
    merkle_root = s.read(32)[::-1]
    timestamp = little_endian_to_int(s.read(4))
    bits = s.read(4)
    nonce = s.read(4)
    return cls(version, prev_block, merkle_root, timestamp, bits, nonce)

### Exercise 4

Write the `serialize` for `Block`.

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_serialize`

In [8]:
# Exercise 4

# reload(block)
# run(block.BlockTest("test_serialize"))
def serialize(self):
    result = int_to_little_endian(self.version, 4)
    result += self.prev_block[::-1]
    result += self.merkle_root[::-1]
    result += int_to_little_endian(self.timestamp, 4)
    result += self.bits
    result += self.nonce
    return result

### Exercise 5

Write the `hash` for `Block`.

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_hash`

In [9]:
# Exercise 5

# reload(block)
# run(block.BlockTest("test_hash"))
def hash(self):
    s = self.serialize()
    sha = hash256(s)
    return sha[::-1]

In [10]:
from io import BytesIO
from block import Block
b = Block.parse(BytesIO(bytes.fromhex('020000208ec39428b17323fa0ddec8e887b4a7c53b8c0a0a220cfd0000000000000000005b0750fce0a889502d40508d39576821155e9c9e3f5c3157f961db38fd8b25be1e77a759e93c0118a4ffd71d')))
print('BIP9: {}'.format(b.version >> 29 == 0b001))
print('BIP91: {}'.format(b.version >> 4 & 1 == 1))
print('BIP141: {}'.format(b.version >> 1 & 1 == 1))

NotImplementedError: 

### Exercise 6

Write the `bip9` method for the `Block` class.

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_bip9`

In [11]:
# Exercise 6

# reload(block)
# run(block.BlockTest("test_bip9"))
def bip9(self):
    return self.version >> 29 == 0b001

### Exercise 7

Write the `bip91` method for the `Block` class.

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_bip91`

In [12]:
# Exercise 7

# reload(block)
# run(block.BlockTest("test_bip91"))
def bip91(self):
    return self.version >> 4 & 1 == 1

### Exercise 8

Write the `bip141` method for the `Block` class.

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_bip141`

In [13]:
# Exercise 8

# reload(block)
# run(block.BlockTest("test_bip141"))
def bip141(self):
    return self.version >> 1 & 1 == 1

In [14]:
from helper import hash256
block_id = hash256(bytes.fromhex('020000208ec39428b17323fa0ddec8e887b4a7c53b8c0a0a220cfd0000000000000000005b0750fce0a889502d40508d39576821155e9c9e3f5c3157f961db38fd8b25be1e77a759e93c0118a4ffd71d'))[::-1]
print('{}'.format(block_id.hex()).zfill(64))

# return : 0000000000000000007e9e4c586439b0cdbe13b1370bdd9435d76a644d047523

0000000000000000007e9e4c586439b0cdbe13b1370bdd9435d76a644d047523


In [15]:
from helper import little_endian_to_int
bits = bytes.fromhex('e93c0118')
exponent = bits[-1]
coefficient = little_endian_to_int(bits[:-1])
target = coefficient * 256**(exponent - 3)
print('{:x}'.format(target).zfill(64))

# return : 0000000000000000013ce9000000000000000000000000000000000000000000


0000000000000000013ce9000000000000000000000000000000000000000000


In [17]:
from helper import little_endian_to_int
proof = little_endian_to_int(hash256(bytes.fromhex('020000208ec39428b17323fa0ddec8e887b4a7c53b8c0a0a220cfd0000000000000000005b0750fce0a889502d40508d39576821155e9c9e3f5c3157f961db38fd8b25be1e77a759e93c0118a4ffd71d')))
print(proof < target)

# return : True

True


### Exercise 9

Write the `bits_to_target` function in `helper.py`.

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_target`

In [18]:
# Exercise 9

# reload(block)
# run(block.BlockTest("test_target"))
def bits_to_target(bits):
    exponent = bits[-1]
    coefficient = little_endian_to_int(bits[:-1])
    return coefficient * 256**(exponent - 3)

In [20]:
from helper import little_endian_to_int
bits = bytes.fromhex('e93c0118')
exponent = bits[-1]
coefficient = little_endian_to_int(bits[:-1])
target = coefficient*256**(exponent-3)
difficulty = 0xffff * 256**(0x1d-3) / target
print(difficulty)

# return : 888171856257.3206

888171856257.3206


### Exercise 10

Write the `difficulty` method for `Block`

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_difficulty`

In [21]:
# Exercise 10

# reload(block)
# run(block.BlockTest("test_difficulty"))
def difficulty(self):
    lowest = 0xffff * 256**(0x1d - 3)
    return lowest / self.target()

### Exercise 11

Write the `check_pow` method for `Block`.

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_check_pow`

In [22]:
# Exercise 11

# reload(block)
# run(block.BlockTest("test_check_pow"))
def check_pow(self):
    sha = hash256(self.serialize())
    proof = little_endian_to_int(sha)
    return proof < self.target()

In [23]:
from block import Block
from helper import TWO_WEEKS
last_block = Block.parse(BytesIO(bytes.fromhex('00000020fdf740b0e49cf75bb3d5168fb3586f7613dcc5cd89675b0100000000000000002e37b144c0baced07eb7e7b64da916cd3121f2427005551aeb0ec6a6402ac7d7f0e4235954d801187f5da9f5')))
first_block = Block.parse(BytesIO(bytes.fromhex('000000201ecd89664fd205a37566e694269ed76e425803003628ab010000000000000000bfcade29d080d9aae8fd461254b041805ae442749f2a40100440fc0e3d5868e55019345954d80118a1721b2e')))
time_differential = last_block.timestamp - first_block.timestamp
if time_differential > TWO_WEEKS * 4:
    time_differential = TWO_WEEKS * 4
if time_differential < TWO_WEEKS // 4:
    time_differential = TWO_WEEKS // 4
new_target = last_block.target() * time_differential // TWO_WEEKS
print('{:x}'.format(new_target).zfill(64))

NotImplementedError: 

### Exercise 12

Calculate the new bits given the first and last blocks of this 2016 block difficulty adjustment period:

Block 471744:

```
000000203471101bbda3fe307664b3283a9ef0e97d9a38a7eacd8800000000000000000010c8ab
a8479bbaa5e0848152fd3c2289ca50e1c3e58c9a4faaafbdf5803c5448ddb845597e8b0118e43a
81d3
```

Block 473759:

```
02000020f1472d9db4b563c35f97c428ac903f23b7fc055d1cfc26000000000000000000b3f449
fcbe1bc4cfbcb8283a0d2c037f961a3fdf2b8bedc144973735eea707e1264258597e8b0118e5f0
0474
```

In [29]:
# Exercise 12
from io import BytesIO
from block import Block
from helper import TWO_WEEKS
from helper import target_to_bits

block1_hex = '000000203471101bbda3fe307664b3283a9ef0e97d9a38a7eacd8800000000000000000010c8aba8479bbaa5e0848152fd3c2289ca50e1c3e58c9a4faaafbdf5803c5448ddb845597e8b0118e43a81d3'
block2_hex = '02000020f1472d9db4b563c35f97c428ac903f23b7fc055d1cfc26000000000000000000b3f449fcbe1bc4cfbcb8283a0d2c037f961a3fdf2b8bedc144973735eea707e1264258597e8b0118e5f00474'

last_block = Block.parse(BytesIO(bytes.fromhex(block1_hex)))
first_block = Block.parse(BytesIO(bytes.fromhex(block2_hex)))
time_differential = last_block.timestamp - first_block.timestamp
if time_differential > TWO_WEEKS * 4:time_differential = TWO_WEEKS * 4
if time_differential < TWO_WEEKS // 4:time_differential = TWO_WEEKS // 4
new_target = last_block.target() * time_differential // TWO_WEEKS
new_bits = target_to_bits(new_target)
print(new_bits.hex())

# return : 80df6217



NotImplementedError: 

### Exercise 13

Write the `calculate_new_bits` function in `helper.py`

#### Make [this test](/edit/code-ch09/helper.py) pass: `helper.py:HelperTest:test_calculate_new_bits`

In [32]:
# Exercise 13

# reload(helper)
# run(helper.HelperTest("test_calculate_new_bits"))
def calculate_new_bits(previous_bits, time_differential):
    if time_differential > TWO_WEEKS * 4:
        time_differential = TWO_WEEKS * 4
    if time_differential < TWO_WEEKS // 4:
        time_differential = TWO_WEEKS // 4
    new_target = bits_to_target(previous_bits) * time_differential // TWO_WEEKS
    return target_to_bits(new_target)